<a href="https://colab.research.google.com/github/Mushtariy00/Mushtariy00/blob/main/Training_custom_dataset_for_Custom_Object_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/khanhlvg/tflite_raspberry_pi/blob/main/object_detection/Train_custom_model_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Train a custom object detection model with TensorFlow Lite Model Maker

In this colab notebook, we use the TensorFlow Lite Model Maker to train a custom object detection model and use it on Rapberry Pi.
We use exsisting trained model to train our model with our custom dataset. This method is called  *transfer learning*. Retraining a exsisting model with your own custom dataset reduces the amount of training data and will shorten the training time.


## Preparation

### Install the required packages


In [1]:
!pip install -q tflite-model-maker
!pip install -q tflite-support

     |████████████████████████████████| 616 kB 13.1 MB/s 
     |████████████████████████████████| 1.1 MB 45.6 MB/s 
     |████████████████████████████████| 3.4 MB 42.3 MB/s 
     |████████████████████████████████| 120 kB 50.6 MB/s 
     |████████████████████████████████| 87 kB 6.1 MB/s 
     |████████████████████████████████| 237 kB 48.1 MB/s 
     |████████████████████████████████| 1.1 MB 36.8 MB/s 
     |████████████████████████████████| 596 kB 49.7 MB/s 
     |████████████████████████████████| 77 kB 5.1 MB/s 
     |████████████████████████████████| 840 kB 52.6 MB/s 
     |████████████████████████████████| 6.4 MB 43.4 MB/s 
     |████████████████████████████████| 1.2 MB 44.5 MB/s 
     |████████████████████████████████| 25.3 MB 10.7 MB/s 
     |████████████████████████████████| 47.8 MB 52 kB/s 
     |████████████████████████████████| 99 kB 8.8 MB/s 
     |████████████████████████████████| 352 kB 53.1 MB/s 
     |████████████████████████████████| 462 kB 55.7 MB/s 
     |██████████████

### Import the required packages.

In [2]:
import numpy as np
import os

from tflite_model_maker.config import ExportFormat, QuantizationConfig
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

from tflite_support import metadata

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

### Connect google drive to Colab then you can easily use datasets in your goole drive. 

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Unzip the dataset file and extract the data

In [4]:
!unzip -q /content/drive/My\ Drive/object_detection/data/PE.zip

## Train the object detection model

### Step 1: Load the dataset

* Images in `train_data` is used to train the custom object detection model.
* Images in `val_data` is used to check if the model can generalize well to new images that it hasn't seen before.

In [5]:
train_data = object_detector.DataLoader.from_pascal_voc(
    'PE/train',
    'PE/train',
    ['pig', 'elephant']
)

val_data = object_detector.DataLoader.from_pascal_voc(
    'PE/validate',
    'PE/validate',
    ['pig', 'elephant']
)

### Step 2: Select a model architecture

EfficientDet-Lite[0-4] are a family of mobile/IoT-friendly object detection models derived from the [EfficientDet](https://arxiv.org/abs/1911.09070) architecture.

Here is the performance of each EfficientDet-Lite models compared to each others.

| Model architecture | Size(MB)* | Latency(ms)** | Average Precision*** |
|--------------------|-----------|---------------|----------------------|
| EfficientDet-Lite0 | 4.4       | 146           | 25.69%               |
| EfficientDet-Lite1 | 5.8       | 259           | 30.55%               |
| EfficientDet-Lite2 | 7.2       | 396           | 33.97%               |
| EfficientDet-Lite3 | 11.4      | 716           | 37.70%               |
| EfficientDet-Lite4 | 19.9      | 1886          | 41.96%               |


In this notebook, we use EfficientDet-Lite0 to train our model. You can choose other model architectures depending on whether speed or accuracy is more important to you.

In [6]:
spec = model_spec.get('efficientdet_lite0')

### Step 3: Train the TensorFlow model with the training data.

* Set `epochs = 50`, our custom model will be trained by using the same custom dataset 50 times over and over. 
* Set `batch_size = 10` here so you will see that it takes 18 steps to go through the 180 images in the training dataset.
* Set `train_whole_model=True` to retrain the whole model instead of just training the head layer to improve accuracy.

In [7]:
model = object_detector.create(train_data, model_spec=spec, batch_size=10, train_whole_model=True, epochs=50, validation_data=val_data)

Epoch 1/50
18/18 [==============================] - 70s 1s/step - det_loss: 1.6944 - cls_loss: 1.1278 - box_loss: 0.0113 - reg_l2_loss: 0.0629 - loss: 1.7574 - learning_rate: 0.0102 - gradient_norm: 1.3756 - val_det_loss: 1.4489 - val_cls_loss: 1.0186 - val_box_loss: 0.0086 - val_reg_l2_loss: 0.0629 - val_loss: 1.5118
Epoch 2/50
18/18 [==============================] - 16s 877ms/step - det_loss: 1.2556 - cls_loss: 0.8592 - box_loss: 0.0079 - reg_l2_loss: 0.0630 - loss: 1.3185 - learning_rate: 0.0125 - gradient_norm: 2.0625 - val_det_loss: 1.4003 - val_cls_loss: 1.0971 - val_box_loss: 0.0061 - val_reg_l2_loss: 0.0630 - val_loss: 1.4633
Epoch 3/50
18/18 [==============================] - 16s 897ms/step - det_loss: 0.8078 - cls_loss: 0.5440 - box_loss: 0.0053 - reg_l2_loss: 0.0630 - loss: 0.8708 - learning_rate: 0.0124 - gradient_norm: 2.0619 - val_det_loss: 0.8604 - val_cls_loss: 0.4959 - val_box_loss: 0.0073 - val_reg_l2_loss: 0.0630 - val_loss: 0.9234
Epoch 4/50
18/18 [================

### Step 4. Evaluate the model with the validation data.

After training the object detection model using the images in the training dataset, we use the images in the validation dataset to evaluate how the model performs against new data it has never seen before.



In [8]:
model.evaluate(val_data)

1/1 [==============================] - 9s 9s/step



{'AP': 0.9370231,
 'AP50': 1.0,
 'AP75': 1.0,
 'AP_/elephant': 0.9241915,
 'AP_/pig': 0.9498548,
 'APl': 0.9370493,
 'APm': -1.0,
 'APs': -1.0,
 'ARl': 0.965625,
 'ARm': -1.0,
 'ARmax1': 0.946875,
 'ARmax10': 0.965625,
 'ARmax100': 0.965625,
 'ARs': -1.0}

### Step 5: Export as a TensorFlow Lite model.

Export the trained object detection model to the TensorFlow Lite format by specifying which folder you want to export.

In [9]:
model.export(export_dir='.', tflite_filename='my_model.tflite')

### Download the TensorFlow Lite model
Download the TFLite model to your local computer to use on Raspberry Pi for detecting your custom obejects.


In [11]:
from google.colab import files
files.download('my_model.tflite')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>